In [1]:
import os
from datetime import datetime
from typing import Dict, Tuple, Any
from tqdm import tqdm
import pickle

import math
import numpy as np
import pandas as pd

from scipy.special import softmax
from sklearn.model_selection import train_test_split, StratifiedKFold

import cv2
import albumentations
from torch.utils.data import Dataset

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch.autograd import Variable
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import OneCycleLR, ReduceLROnPlateau

import timm

import psutil
def get_used_memory():
    return psutil.Process(os.getpid()).memory_info().vms/1024**3
def get_used_memory_txt():
    return 'Used memory: {:.2f}'.format(get_used_memory())
initial_used_memory=get_used_memory()
print(get_used_memory_txt())

Used memory: 2.96


In [2]:
# ---------------------------------------
# parameters

MODEL_DIR = './model_checkpoints/'
DATA_DIR = '../input/'
LOG_DIR = './logs/'
DEVICE = 'cuda:0'
MODEL_NAME = 'rexnet_200'

TRAIN_STEP = 0
FOLD = 0

IMAGE_SIZE = 256
BATCH_SIZE = 64
NUM_EPOCHS = 10
NUM_WORKERS = 4
LR = 1e-4
USE_AMP = True


In [3]:
PICKLE_PROTOCOL=4  # there are issues with pickle.HIGHEST_PROTOCOL unsing local recent python versions and then kaggle kernels

def read_pickle(filename):
    ### LOAD PICKLE
    with open(filename, "rb") as pfile:
        x = pickle.load(pfile)
    return x

def save_pickle(x, filename=None, makedirs=True):
    if makedirs and os.path.dirname(filename)!='': os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(filename, 'wb') as pfile:
        pickle.dump(x, pfile, protocol=PICKLE_PROTOCOL)

def to_hex_id(i):
    #return i.to_bytes(((i.bit_length() + 7) // 8),"big").hex(), 
    #return hex(i)[2:]
    return format(i, '#018x')[2:]

def to_int_id(id_):
    return int(id_, 16)

adversarials_dict=read_pickle("../input/adversarials.pkl")
best_friends_dict=read_pickle("../input/best_friends.pkl")

In [4]:
def get_closer_images(id_, d, top_k=None, threshold=None, return_type='hex', include_query_in_result=True):
    
    if isinstance(id_, str): id_ = to_int_id(id_)
    
    ids, vals = d[id_]
    
    if not include_query_in_result:
        sel = [x!=id_ for x in ids]
        ids = ids[sel]
        vals = vals[sel]
        
    if top_k is not None:
        ids = ids[:top_k]
        vals = vals[:top_k]
    
    if threshold is not None:
        sel = [x>=threshold for x in vals]
        ids = ids[sel]
        vals = vals[sel]
    
    if return_type == 'hex':
        ids = [to_hex_id(x) for x in ids]
    
    return ids, vals

In [5]:
df_train = pd.read_csv(os.path.join(DATA_DIR, 'train.csv'))

# num_imgs = 600_000

# list_records = df_train[['id', 'landmark_id']].to_dict('records')
# np.random.shuffle(list_records)
# list_records_sample = list_records[:num_imgs]
# list_ids_train = df_train['id'].to_list()

In [6]:
num_bf_per_query = 3
num_adv_per_query = 4
list_id, list_id_crossed = [], []
list_same_landmark = []
arr_ids = df_train.id.apply(lambda x: to_int_id(x)).values

In [35]:
for id_ in tqdm(arr_ids):
    
    # Best Friends
    qt = len(best_friends_dict[id_][1][1:])
    if qt > 0:
        if qt < num_bf_per_query:
            ids_bf = np.random.choice(best_friends_dict[id_][0][1:], qt, p=softmax(best_friends_dict[id_][1][1:]))
        else:
            ids_bf = np.random.choice(best_friends_dict[id_][0][1:], num_bf_per_query, p=softmax(best_friends_dict[id_][1][1:]))
    else:
        ids_bf = [] 
    
    # Adversarials
    qt = len(adversarials_dict[id_][1][1:])
    if qt > 0:
        if qt < num_bf_per_query:
            ids_adv = np.random.choice(adversarials_dict[id_][0][1:], qt, p=softmax(adversarials_dict[id_][1][1:]))
        else:
            ids_adv = np.random.choice(adversarials_dict[id_][0][1:], num_bf_per_query, p=softmax(adversarials_dict[id_][1][1:]))
    else:
        ids_adv = []
    
    ids_bf = [to_hex_id(x) for x in ids_bf]
    ids_adv = [to_hex_id(x) for x in ids_adv]
    
    list_id.append(np.repeat(id_, len(ids_bf)+len(ids_adv)))
    list_id_crossed.append(ids_bf + ids_adv)
    list_same_landmark.extend([1 for _ in range(len(ids_bf))] + [0 for _ in range(len(ids_adv))])
    
df_vector_discriminator = pd.DataFrame({
    'img_id' : np.concatenate(list_id),
    'img_id_crossed' : np.concatenate(list_id_crossed),
    'target' : list_same_landmark
})

100%|██████████| 1580470/1580470 [02:56<00:00, 8958.92it/s] 


In [48]:
df_vector_discriminator['img_id'] = df_vector_discriminator['img_id'].apply(lambda x: to_hex_id(x))
imgid2lmid = df_train.to_dict('records')
imgid2lmid = {d['id']: d['landmark_id'] for d in imgid2lmid}
df_vector_discriminator['landmark_id'] = df_vector_discriminator['img_id'].apply(lambda x: imgid2lmid[x])

In [81]:
lmid2class = {id_: i for i, id_ in enumerate(sorted(df_vector_discriminator['landmark_id'].unique()))}

In [83]:
df_vector_discriminator['landmark_class'] = df_vector_discriminator['landmark_id'].apply(lambda x: lmid2class[x])

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
df_vector_discriminator['fold'] = -1

for i, (_, vld_idx) in enumerate(skf.split(df_vector_discriminator.index, df_vector_discriminator['landmark_class'])):
    df_vector_discriminator['fold'].iloc[vld_idx] = i

In [85]:
df_vector_discriminator.to_csv('../input/train_adversarials.csv', index=False)

In [84]:
df_vector_discriminator

,img_id,img_id_crossed,target,landmark_id,landmark_class
0,17660ef415d37059,af82f37db037efd0,1,1,0
1,17660ef415d37059,183180bf74cd6933,1,1,0
2,17660ef415d37059,878fb29295c7ae0e,1,1,0
3,17660ef415d37059,b12506b7608ab2e4,0,1,0
4,17660ef415d37059,f62cdcd356702122,0,1,0
...,...,...,...,...,...
9354300,d9e338c530dca106,5ba296fb81175d30,1,203092,81312
9354301,d9e338c530dca106,9401fad4c497e1f9,1,203092,81312
9354302,d9e338c530dca106,5a7934544b8cea76,0,203092,81312
9354303,d9e338c530dca106,275839a063cb77b7,0,203092,81312


In [86]:
embeddings = np.load('../input/model_v0.6_landmark_embeddings_f16.npy')

In [89]:
df_train

,id,landmark_id
0,17660ef415d37059,1
1,92b6290d571448f6,1
2,cd41bf948edc0340,1
3,fb09f1e98c6d2f70,1
4,25c9dfc7ea69838d,7
...,...,...
1580465,72c3b1c367e3d559,203092
1580466,7a6a2d9ea92684a6,203092
1580467,9401fad4c497e1f9,203092
1580468,aacc960c9a228b5f,203092


In [87]:
embeddings.shape

(1580470, 512)

In [43]:
class Swish(torch.autograd.Function):

    @staticmethod
    def forward(ctx, i):
        result = i * torch.sigmoid(i)
        ctx.save_for_backward(i)
        return result

    @staticmethod
    def backward(ctx, grad_output):
        i = ctx.saved_variables[0]
        sigmoid_i = torch.sigmoid(i)
        return grad_output * (sigmoid_i * (1 + i * (1 - sigmoid_i)))


class Swish_module(nn.Module):
    @autocast()
    def forward(self, x):
        return Swish.apply(x)

def gem(x, p=3, eps=1e-6):
    return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1./p)

class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6, p_trainable=True):
        super(GeM,self).__init__()
        if p_trainable:
            self.p = Parameter(torch.ones(1)*p)
        else:
            self.p = p
        self.eps = eps

    def forward(self, x):
        return gem(x, p=self.p, eps=self.eps)
    
    def __repr__(self):
        return self.__class__.__name__ + '(' + 'p=' + '{:.4f}'.format(self.p.data.tolist()[0]) + ', ' + 'eps=' + str(self.eps) + ')'

class Landmark_Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        #self.backbone = timm.create_model('rexnet_100', pretrained=True)
        self.backbone = timm.create_model('res2net50_14w_8s', pretrained=True)
        self.global_pool = GeM()
        self.neck = nn.Sequential(
            nn.Linear(self.backbone.num_features, 512, bias=True),
            nn.BatchNorm1d(512),
            Swish_module()
        )
        self.backbone.global_pool = nn.Identity()
        self.backbone.fc = nn.Identity()
        self.head = nn.Sequential(nn.Linear(512*3, 1, bias=True))
        
    def forward(self, x0, x1):
        embd0 = self.neck(self.global_pool(self.backbone(x0))[:, :, 0, 0])
        embd1 = self.neck(self.global_pool(self.backbone(x1))[:, :, 0, 0])
        embd2 = (embd1 - embd0).abs()
        embd = torch.cat([embd0, embd1, embd2], dim=1)
        return self.head(embd)

In [44]:
m = Landmark_Discriminator()

In [45]:
m(torch.randn(8, 3, 512, 512), torch.randn(8, 3, 512, 512))

tensor([[-0.2629],
        [-0.8639],
        [-0.5148],
        [-0.4444],
        [-0.9016],
        [-0.4894],
        [-0.0449],
        [-0.4814]], grad_fn=<AddmmBackward>)